In [1]:
import pandas as pd
import os

In [32]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    outlist = []
    for i in range(0, len(lst), n):
        outlist.append(lst[i:i + n])
    return outlist





# def write_sbatch(idat_list, script_path, job_name, log_out, log_err, ntasks, nodes, time, array):
    
#     # Create a sbatch script
#     with open(f'{script_path}', 'w') as f:
#         f.write('#!/bin/bash\n')
#         f.write(f'#SBATCH --job-name={job_name}\n')
#         f.write(f'#SBATCH --output={log_out}\n')
#         f.write(f'#SBATCH --error={log_err}\n')
#         f.write(f'#SBATCH --ntasks={ntasks}\n')
#         f.write(f'#SBATCH --nodes={nodes}\n')
#         f.write(f'#SBATCH --time={time}\n')
#         f.write(f'#SBATCH --array={array}\n')
#         f.write('\n')
#         f.write('echo "Starting job"\n')
#         f.write('date\n')
#         f.write('\n')

#         # for i, idat_path in enumerate(idat_list):
#         for code in idat_list:
            
#             # code = idat_path.split('/')[-1]
#             idat_dir_in = f'{datapath}/gp2_idats/{code}'
#             # plink_dir_out = f'{datapath}/gp2_plink'

#             gcs_path = f'gp2_uk'
#             gcs_idat_path = f'{gcs_path}/gp2_idats/{code}'
#             gcs_plink_path = f'{gcs_path}/gp2_plink'
            
#             # files not writing properly to gcs via gcsfuse so write to path in vm and copy to gcs 
#             tmp_out_dir = f'~/tmp/{code}'

#             command = f'\
#     cd ~/; mkdir {idat_dir_in}; \
#     gcsfuse --dir-mode 776 --file-mode 776 --implicit-dirs {gcs_idat_path} {idat_dir_in}; \
#     {iaap} gencall \
#     {bpm} \
#     {egt} \
#     {tmp_out_dir} \
#     -f {idat_dir_in} \
#     -p \
#     -t 2; \
#     gsutil cp -r {tmp_out_dir}/ gs://{gcs_plink_path}/'
#             f.write(f'# Run command for file {idat_dir_in}\n')
#             f.write(f'{command}\n')
#         f.write('\n')
#         f.write('echo "Job finished"\n')
#         f.write('date\n')
#     f.close()



In [33]:
# cluster config variables
max_nodes = 10

# list of idats in directory of choice
# !gsutil ls gs://gp2_uk/gp2_idats/ > tmp/idats.txt
idats_in = pd.read_csv('tmp/idats.txt', header=None)
idat_list = idats_in.loc[1:,0]
idat_list = [x.replace('gs://gp2_uk/gp2_idats/','')[:-1] for x in idat_list]

# chunk list by max nodes
idat_list_chunks = chunks(idat_list, max_nodes)

# paths
datapath = '~/data'
tmp_dir = '~/tmp'
ilmn_files_path = '~/ilmn_files'
iaap = '~/GenoTools/executables/iaap-cli-linux-x64-1.1.0-sha.80d7e5b3d9c1fdfc2e99b472a90652fd3848bbc7/iaap-cli/iaap-cli'
bpm = f'{datapath}/ilmn_files/NeuroBooster_20042459_A1.bpm'
egt = f'{datapath}/ilmn_files/recluster_09272022.egt'
plink_file_path = f'{datapath}/ilmn_files/gp2_plink'

In [43]:
# now loop through chunks to create a job for every {max_nodes} barcodes 
# 1 node per barcode. (n_idats<=(80*max_nodes))

for i, chunk in enumerate(idat_list_chunks):

    script_path = f'cluster_scripts/call_gts_{i}.sh'
    job_name = f'callgts_{i}'
    log_path = '/home/dan_datatecnica_com/logs'
    log_out = f'{log_path}/test.out'
    log_err = f'{log_path}/test.err'
    # log_out = f'{log_path}/{job_name}.%A_%a.out'
    # log_err = f'{log_path}/{job_name}.%A_%a.err'
    # ntasks = str(len(idat_list))
    # nodes = str(len(idat_list))
    nodes = len(chunk)
    ntasks = len(chunk)
    time = '01:00:00'
    array = f'1-{len(chunk)}'

    # Create a sbatch script
    with open(f'cluster_scripts/call_gts_{i}.sh', 'w') as f:
        f.write('#!/bin/bash\n')
        f.write(f'#SBATCH --job-name={job_name}\n')
        f.write(f'#SBATCH --output={log_out}\n')
        f.write(f'#SBATCH --error={log_err}\n')
        f.write(f'#SBATCH --ntasks={ntasks}\n')
        f.write(f'#SBATCH --nodes={nodes}\n')
        f.write(f'#SBATCH --time={time}\n')
        f.write(f'#SBATCH --array={array}\n')
        f.write('\n')
        f.write('echo "Starting job"\n')
        f.write('date\n')
        f.write('\n')

        for code in idat_list:
            
            # path in vm that is mounted to bucket
            idat_dir_in = f'{datapath}/gp2_idats/{code}'

            # bucket paths
            gcs_path = f'gp2_uk'
            gcs_idat_path = f'{gcs_path}/gp2_idats/{code}'
            gcs_plink_path = f'{gcs_path}/gp2_plink'
            
            # files not writing properly to gcs via gcsfuse so write to path in vm and copy to gcs 
            tmp_out_dir = f'~/tmp/{code}'

            command = f'\
cd ~/; mkdir {idat_dir_in}; \
gcsfuse --dir-mode 776 --file-mode 776 --implicit-dirs {gcs_idat_path} {idat_dir_in}; \
{iaap} gencall \
{bpm} \
{egt} \
{tmp_out_dir} \
-f {idat_dir_in} \
-p \
-t 2; \
gsutil cp -r {tmp_out_dir}/ gs://{gcs_plink_path}/'

            f.write(f'# Run command for file {idat_dir_in}\n')
            f.write(f'{command}\n')
            f.write('\n')

        f.write('\n')
        f.write('echo "Job finished"\n')
        f.write('date\n')
    f.close()

In [44]:
# copy scripts to vm
!gcloud compute scp cluster_scripts/call_gts_*.sh test-login0:/home/dan_datatecnica_com/scripts/ --project genotools --zone europe-west2-a

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
call_gts_0.sh                                 100% 6759    50.5KB/s   00:00    
call_gts_1.sh                                 100% 6756    50.1KB/s   00:00    


In [45]:
# launch commands
!gcloud compute ssh --zone europe-west2-a --project genotools test-login0 --command 'sbatch ~/scripts/call_gts_0.sh'

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
Submitted batch job 21


In [37]:
!cat cluster_scripts/call_gts_{i}.sh


#!/bin/bash
#SBATCH --job-name=callgts_1
#SBATCH --output=~/logs/callgts_1.%A_%a.out
#SBATCH --error=~/logs/callgts_1.%A_%a.err
#SBATCH --ntasks=2
#SBATCH --nodes=2
#SBATCH --time=01:00:00
#SBATCH --array=1-2

echo "Starting job"
date

# Run command for file ~/data/gp2_idats/204620380001
cd ~/; mkdir ~/data/gp2_idats/204620380001; gcsfuse --dir-mode 776 --file-mode 776 --implicit-dirs gp2_uk/gp2_idats/204620380001 ~/data/gp2_idats/204620380001; ~/GenoTools/executables/iaap-cli-linux-x64-1.1.0-sha.80d7e5b3d9c1fdfc2e99b472a90652fd3848bbc7/iaap-cli/iaap-cli gencall ~/data/ilmn_files/NeuroBooster_20042459_A1.bpm ~/data/ilmn_files/recluster_09272022.egt ~/tmp/204620380001 -f ~/data/gp2_idats/204620380001 -p -t 2; gsutil cp -r ~/tmp/204620380001/ gs://gp2_uk/gp2_plink/

# Run command for file ~/data/gp2_idats/206412050154
cd ~/; mkdir ~/data/gp2_idats/206412050154; gcsfuse --dir-mode 776 --file-mode 776 --implicit-dirs gp2_uk/gp2_idats/206412050154 ~/data/gp2_idats/206412050154; ~/GenoTools/

In [17]:
idat_list_chunks

<generator object chunks at 0x157aa2190>

In [19]:
for i, chunk in enumerate(idat_list_chunks):
    print(i)

In [23]:
import pprint
pprint.pprint(list(idat_list_chunks))

[]


In [24]:
idat_list

['204620380001',
 '206412050154',
 '206412050160',
 '206412050162',
 '206412050175',
 '206451070064',
 '206451070070',
 '206451070071',
 '206451070072',
 '206451070101',
 '206451070115',
 '206451070117']